In [1]:
import torch 
import math 
import numpy as np
import copy
from torch import nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import ast 
from numpy import load
import torch.nn as nn
import random
import time
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [2]:
import collections
#import helper
import numpy as np
#import project_tests as tests
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [3]:
import torch.nn as nn

class Embedder(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model)
    def forward(self, x):
        return self.embed(x)

In [4]:
class PositionalEncoder(nn.Module):
    def __init__(self, d_model, max_seq_len = 120):
        super().__init__()
        self.d_model = d_model
        
        # create constant 'pe' matrix with values dependant on 
        # pos and i
        pe = torch.zeros(max_seq_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
                
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
 
    
    def forward(self, x):
        # make embeddings relatively larger
        x = x * math.sqrt(self.d_model)
        #add constant to embedding
        seq_len = x.size(1)
        x = x + Variable(self.pe[:,:seq_len], \
        requires_grad=False).cuda(1)
        return x

In [5]:
import torch
from   core_qnn.quaternion_layers       import QuaternionLinearAutograd

In [6]:
class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout = 0.1):
        super().__init__()
        
        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads
        
        self.q_linear = QuaternionLinearAutograd(d_model, d_model)
        self.v_linear = QuaternionLinearAutograd(d_model, d_model)
        self.k_linear = QuaternionLinearAutograd(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.out = QuaternionLinearAutograd(d_model, d_model)
    
    def forward(self, q, k, v, mask=None):
        
        bs = q.size(0)
        
        # perform linear operation and split into h heads
        
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)
        
        # transpose to get dimensions bs * h * sl * d_model
       
        k = k.transpose(1,2)
        q = q.transpose(1,2)
        v = v.transpose(1,2)
        # calculate attention using function we will define next
        
        #print(k.shape,q.shape,v.shape)
        scores = attention(q, k, v, self.d_k, mask, self.dropout)
        
        # concatenate heads and put through final linear layer
        concat = scores.transpose(1,2).contiguous()\
        .view(bs, -1, self.d_model)
        
        output = self.out(concat)
    
        return output

In [7]:
def quarternion_multiplication(a, b, transpose=True):
    """ Performs hamilton product between two quarternion sequences.
    a = (r,x,y,z)
    b = (r',x',y',z')
    following:
    (rr' - xx' - yy' - zz')  +
    (rx' + xr' + yz' - zy')i +
    (ry' - xz' + yr' + zx')j +
    (rz' + xy' - yx' + zr')k
    """


    ar, ax, ay, az = torch.chunk(a, chunks=4, dim=-1)
    br, bx, by, bz = torch.chunk(b, chunks=4, dim=-1)
    #print(ar.shape)
    #print(br.shape)

    if transpose==True:
        if len(br.shape)>2:
            #r = torch.matmul(br.transpose(-2,-1),ar) - torch.matmul(bx.transpose(-2,-1),ax) - torch.matmul(by.transpose(-2,-1),ay) - torch.matmul(bz.transpose(-2,-1),az)
            #i = torch.matmul(bx.transpose(-2,-1),ar) + torch.matmul(br.transpose(-2,-1),ax) + torch.matmul(bz.transpose(-2,-1),ay) - torch.matmul(by.transpose(-2,-1),az)
            #j = torch.matmul(by.transpose(-2,-1),ar) - torch.matmul(bz.transpose(-2,-1),ax) + torch.matmul(br.transpose(-2,-1),ay) + torch.matmul(bx.transpose(-2,-1),az)
            #k = torch.matmul(bz.transpose(-2,-1),ar) + torch.matmul(by.transpose(-2,-1),ax) - torch.matmul(bx.transpose(-2,-1),ay) + torch.matmul(br.transpose(-2,-1),az)
        
            r = torch.matmul(ar,br.transpose(-2,-1)) - torch.matmul(ax,bx.transpose(-2,-1)) - torch.matmul(ay,by.transpose(-2,-1)) - torch.matmul(az,bz.transpose(-2,-1))
            i = torch.matmul(ar,bx.transpose(-2,-1)) + torch.matmul(ax,br.transpose(-2,-1)) + torch.matmul(ay,bz.transpose(-2,-1)) - torch.matmul(az,by.transpose(-2,-1))
            j = torch.matmul(ar,by.transpose(-2,-1)) - torch.matmul(ax,bz.transpose(-2,-1)) + torch.matmul(ay,br.transpose(-2,-1)) + torch.matmul(az,bx.transpose(-2,-1))
            k = torch.matmul(ar,bz.transpose(-2,-1)) + torch.matmul(ax,by.transpose(-2,-1)) - torch.matmul(ay,bx.transpose(-2,-1)) + torch.matmul(az,br.transpose(-2,-1))
        
            
        else:
            r = torch.matmul(ar, br.t()) - torch.matmul(ax, bx.t()) - torch.matmul(ay, by.t()) - torch.matmul(az, bz.t())
            i = torch.matmul(ar, bx.t()) + torch.matmul(ax, br.t()) + torch.matmul(ay, bz.t()) - torch.matmul(az, by.t())
            j = torch.matmul(ar, by.t()) - torch.matmul(ax, bz.t()) + torch.matmul(ay, br.t()) + torch.matmul(az, bx.t())
            k = torch.matmul(ar, bz.t()) + torch.matmul(ax, by.t()) - torch.matmul(ay, bx.t()) + torch.matmul(az, br.t())
    else:
        r = torch.matmul(ar,br) - torch.matmul(ax,bx) - torch.matmul(ay,by) - torch.matmul(az,bz)
        i = torch.matmul(ar,bx) + torch.matmul(ax,br) + torch.matmul(ay,bz) - torch.matmul(az,by)
        j = torch.matmul(ar,by) - torch.matmul(ax,bz) + torch.matmul(ay,br) + torch.matmul(az,bx)
        k = torch.matmul(ar,bz) + torch.matmul(ax,by) - torch.matmul(ay,bx) + torch.matmul(az,br)
        
    return [r, i, j, k]

In [8]:
def attention(q,k, v, d_k, mask=None, dropout=None):
    [scores_r,scores_i,scores_j,scores_k] = [x/math.sqrt(d_k) for x in quarternion_multiplication(q,k)]
    
    if mask is not None:
        #print("mask",mask)
        mask = mask.unsqueeze(1)
        
        #print(scores_r.shape)
        scores_r = scores_r.masked_fill(mask == 0, -1e9)
        scores_r = F.softmax(scores_r, dim=-1)
        scores_i = scores_i.masked_fill(mask == 0, -1e9)
        scores_i = F.softmax(scores_i, dim=-1)
        scores_j = scores_j.masked_fill(mask == 0, -1e9)
        scores_j = F.softmax(scores_j, dim=-1)
        scores_k = scores_k.masked_fill(mask == 0, -1e9)
        scores_k = F.softmax(scores_k, dim=-1)
    
    if dropout is not None:
        scores_r = dropout(scores_r)
        scores_i = dropout(scores_i)
        scores_j = dropout(scores_j)
        scores_k = dropout(scores_k)
    
    scores = torch.cat([scores_r,scores_i,scores_j,scores_k],dim=-1)
    #print(scores.shape)
    #print(v.shape)
    output = quarternion_multiplication(scores,v,transpose=False)
    output = torch.cat(output, dim=-1)
    return output

In [9]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=2048, dropout = 0.1):
        super().__init__() 
        # We set d_ff as a default to 2048
        self.linear_1 = QuaternionLinearAutograd(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = QuaternionLinearAutograd(d_ff, d_model)
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x

In [10]:
def QNorm(x,eps):
    r,i,j,k = torch.chunk(x, chunks=4, dim=-1)
    qnorm = torch.sqrt(r*r + i*i + j*j + k*k + eps)
    r = r/qnorm
    i = i/qnorm
    j = j/qnorm
    k = k/qnorm
        
    return [r,i,j,k]

In [11]:
class Norm(nn.Module):
    def __init__(self, d_model, eps = 1e-6):
        super().__init__()
    
        self.size = d_model//4
        # create two learnable parameters to calibrate normalisation
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        self.eps = eps
    def forward(self, x):
        
        [r,i,j,k] = QNorm(x,self.eps)

        norm_r = self.alpha *r + self.bias
        norm_i = self.alpha *i + self.bias
        norm_j = self.alpha *j + self.bias
        norm_k = self.alpha *k + self.bias
        norm = torch.cat([norm_r,norm_i,norm_j,norm_k],dim=-1)

        return norm
    
    

In [12]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout = 0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.attn = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        x2 = self.norm_1(x)
        
        x = x + self.dropout_1(self.attn(x2,x2,x2,mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.ff(x2))
        return x
    
# build a decoder layer with two multi-head attention layers and
# one feed-forward layer
class DecoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout=0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.norm_3 = Norm(d_model)
        
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        self.dropout_3 = nn.Dropout(dropout)
        
        self.attn_1 = MultiHeadAttention(heads, d_model)
        self.attn_2 = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model).cuda(1)

    def forward(self, x, e_outputs, src_mask, trg_mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn_1(x2, x2, x2, trg_mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.attn_2(x2, e_outputs, e_outputs,
        src_mask))
        x2 = self.norm_3(x)
        x = x + self.dropout_3(self.ff(x2))
        return x
        # We can then build a convenient cloning function that can generate multiple layers:
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

In [13]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.layers = get_clones(EncoderLayer(d_model, heads), N)
        self.norm = Norm(d_model)
    def forward(self, src, mask):
        x = self.embed(src)
        x = self.pe(x)
        for i in range(N):
            x = self.layers[i](x, mask)
        return self.norm(x)
    
class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.layers = get_clones(DecoderLayer(d_model, heads), N)
        self.norm = Norm(d_model)
    def forward(self, trg, e_outputs, src_mask, trg_mask):
        x = self.embed(trg)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, e_outputs, src_mask, trg_mask)
        return self.norm(x)

In [14]:
class Transformer(nn.Module):
    def __init__(self, src_vocab, trg_vocab, d_model, N, heads):
        super().__init__()
        self.encoder = Encoder(src_vocab, d_model, N, heads)
        self.decoder = Decoder(trg_vocab, d_model, N, heads)
        self.out = nn.Linear(d_model, trg_vocab)
    def forward(self, src, trg, src_mask, trg_mask):
        e_outputs = self.encoder(src, src_mask)
        d_output = self.decoder(trg, e_outputs, src_mask, trg_mask)
        output = self.out(d_output)
        return output

In [16]:
"""Prepare the data"""

'Prepare the data'

In [17]:
import pandas as pd 

#test_data=open("/media/data6TB/deepak/NalinIdea/deu-eng/deu.txt")
#daat=[]
#for l in test_data.readlines():
#    daat.append(l.strip().split("\t")[:2])

#test_data.close()

#rest_df = pd.DataFrame(daat[:100000], columns = ['English', 'German'])
#rest_df.to_csv("/media/data6TB/deepak/NalinIdea/deu-eng/new_train_1.csv", index=False)
#test_df = pd.DataFrame(daat[100000:], columns = ['English', 'German'])

import spacy
import torchtext
from torchtext.legacy.data import Field, BucketIterator, TabularDataset

en = spacy.load('en')
de = spacy.load('de')
def tokenize_en(sentence):
    return [tok.text for tok in en.tokenizer(sentence)]
def tokenize_de(sentence):
    return [tok.text for tok in de.tokenizer(sentence)]
EN_TEXT = Field(tokenize=tokenize_en, init_token = "<sos>", eos_token = "<eos>")
DE_TEXT = Field(tokenize=tokenize_de, init_token = "<sos>", eos_token = "<eos>")

from sklearn.model_selection import train_test_split

# create train and validation set 
#train, val = train_test_split(test_df, test_size=0.01)
#train.to_csv("/media/data6TB/deepak/NalinIdea/deu-eng/new_train_2.csv", index=False)
#val.to_csv("/media/data6TB/deepak/NalinIdea/deu-eng/new_val.csv", index=False)

from torchtext.legacy import data
data_fields = [('English', EN_TEXT), ('German', DE_TEXT)]
train,val = data.TabularDataset.splits(path='/media/data6TB/deepak/NalinIdea/deu-eng/', train='new_train_2.csv', validation='new_val.csv', format='csv', fields=data_fields)

DE_TEXT.build_vocab(train, val)
EN_TEXT.build_vocab(train, val)

seq_len=0
for x in train.German:
    if len(x)>seq_len:
        seq_len = len(x)
print(seq_len)


global max_src_in_batch, max_tgt_in_batch
def batch_size_fn(new, count, sofar):
    "Keep augmenting batch and calculate total number of tokens + padding."
    global max_src_in_batch, max_tgt_in_batch
    if count == 1:
        max_src_in_batch = 0
        max_tgt_in_batch = 0
    max_src_in_batch = max(max_src_in_batch,  len(new.English))
    max_tgt_in_batch = max(max_tgt_in_batch,  len(new.German))
    src_elements = count * max_src_in_batch
    tgt_elements = count * max_tgt_in_batch
    return max(src_elements, tgt_elements)
class MyIterator(data.Iterator):
    def create_batches(self):
        if self.train:
            def pool(d, random_shuffler):
                for p in data.batch(d, self.batch_size * 100):
                    p_batch = data.batch(
                        sorted(p, key=self.sort_key),
                        self.batch_size, self.batch_size_fn)
                    for b in random_shuffler(list(p_batch)):
                        yield b
            self.batches = pool(self.data(), self.random_shuffler)
            
        else:
            self.batches = []
            for b in data.batch(self.data(), self.batch_size,
                                          self.batch_size_fn):
                self.batches.append(sorted(b, key=self.sort_key))

train_iter = MyIterator(train, batch_size=64, device=1,
                        repeat=False, sort_key= lambda x:
                        (len(x.English), len(x.German)),
                        batch_size_fn=batch_size_fn, train=True,
                        shuffle=True)

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


85


In [18]:
batch = next(iter(train_iter))
input_seq = batch.English.transpose(0,1)
input_pad = EN_TEXT.vocab.stoi['<pad>']
# creates mask with 0s wherever there is padding in the input
input_msk = (input_seq != input_pad).unsqueeze(1)

from torch.autograd import Variable
import torch
import numpy as np
# create mask as before
target_seq = batch.German.transpose(0,1)
target_pad = DE_TEXT.vocab.stoi['<pad>']
target_msk = (target_seq != target_pad).unsqueeze(1)
size = target_seq.size(1) 
# get seq_len for matrix
nopeak_mask = np.triu(np.ones((1, size, size)),k=1).astype('uint8')
nopeak_mask = Variable(torch.from_numpy(nopeak_mask) == 0)
target_msk = target_msk & nopeak_mask

input_msk = input_msk.cuda(1)
target_msk = target_msk.cuda(1)

In [19]:

def nopeak_mask(size,cuda_enabled):
    np_mask = np.triu(np.ones((1, size, size)),
    k=1).astype('uint8')
    np_mask =  torch.autograd.Variable(torch.from_numpy(np_mask) == 0)

    if cuda_enabled:
        np_mask = np_mask.cuda(1)
    return np_mask

def create_masks(src, trg):
    src_mask = (src != 0).unsqueeze(-2)
    if trg is not None:
        trg_mask = (trg != 0).unsqueeze(-2)
        size = trg.size(1) # get seq_len for matrix
        # print("Sequence lenght in mask ",size)
        np_mask = nopeak_mask(size,True)
        # print(np_mask.shape,trg_mask.shape)
        if trg.is_cuda:
            np_mask.cuda(1)
        trg_mask = trg_mask & np_mask
    else:
        trg_mask = None
    return src_mask, trg_mask

In [21]:
d_model = 80
heads = 4
N = 3
src_vocab = len(EN_TEXT.vocab)
trg_vocab = len(DE_TEXT.vocab)

model = Transformer(src_vocab, trg_vocab, d_model, N, heads)
for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
        
optim = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [22]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
embed_parm = 2*sum([np.prod(p.size()) for p in model.encoder.embed.parameters()])
params,params-embed_parm

(7235545, 4668505)

In [24]:
def train_model(epochs, print_every=100):
    model.cuda(1)
    model.train()

    start = time.time()
    temp = start
    
    total_loss = 0
    
    for epoch in range(epochs):
       
        for i, batch in enumerate(train_iter):            
            src = batch.English.transpose(0,1)
            trg = batch.German.transpose(0,1)            
            
            
            trg_input = trg[:, :-1]
            
            # the words we are trying to predict
            
            targets = trg[:, 1:].contiguous().view(-1)
            
            # create function to make masks using mask code above
            
            if torch.cuda.is_available():
                src = src.cuda(1)
                trg_input = trg_input.cuda(1)
            
            #print(src.is_cuda)    
            #print(trg_input.is_cuda)
            #print(input_msk.is_cuda)
            #print(target_msk.is_cuda)
            
            src_mask, trg_mask = create_masks(src, trg_input)
            #print(src_mask)
            preds = model(src, trg_input, src_mask, trg_mask)
            
            optim.zero_grad()
            
            loss = F.cross_entropy(preds.view(-1, preds.size(-1)),targets.cuda(1), ignore_index=target_pad)            
            loss.backward()
            optim.step()
            
            print(loss.data)
            total_loss += loss.data
            if (i + 1) % print_every == 0:
                loss_avg = total_loss / print_every
                print("time = %dm, epoch %d, iter = %d, loss = %.3f, %ds per %d iters" % ((time.time() - start) // 60, epoch + 1, i + 1, loss_avg, time.time() - temp,print_every))
                total_loss = 0
                temp = time.time()

In [23]:
model

Transformer(
  (encoder): Encoder(
    (embed): Embedder(
      (embed): Embedding(16044, 80)
    )
    (pe): PositionalEncoder()
    (layers): ModuleList(
      (0): EncoderLayer(
        (norm_1): Norm()
        (norm_2): Norm()
        (attn): MultiHeadAttention(
          (q_linear): QuaternionLinearAutograd(in_features=20, out_features=20, bias=True, init_criterion=glorot, weight_init=quaternion, rotation=False, seed=656)
          (v_linear): QuaternionLinearAutograd(in_features=20, out_features=20, bias=True, init_criterion=glorot, weight_init=quaternion, rotation=False, seed=646)
          (k_linear): QuaternionLinearAutograd(in_features=20, out_features=20, bias=True, init_criterion=glorot, weight_init=quaternion, rotation=False, seed=598)
          (dropout): Dropout(p=0.1, inplace=False)
          (out): QuaternionLinearAutograd(in_features=20, out_features=20, bias=True, init_criterion=glorot, weight_init=quaternion, rotation=False, seed=432)
        )
        (ff): FeedFor

In [ ]:
train_model(50, print_every=1)

tensor(10.4393, device='cuda:1')
time = 0m, epoch 1, iter = 1, loss = 10.439, 0s per 1 iters
tensor(10.4145, device='cuda:1')
time = 0m, epoch 1, iter = 2, loss = 10.414, 0s per 1 iters
tensor(10.4077, device='cuda:1')
time = 0m, epoch 1, iter = 3, loss = 10.408, 0s per 1 iters
tensor(10.4190, device='cuda:1')
time = 0m, epoch 1, iter = 4, loss = 10.419, 0s per 1 iters
tensor(10.4143, device='cuda:1')
time = 0m, epoch 1, iter = 5, loss = 10.414, 0s per 1 iters
tensor(10.4188, device='cuda:1')
time = 0m, epoch 1, iter = 6, loss = 10.419, 0s per 1 iters
tensor(10.3822, device='cuda:1')
time = 0m, epoch 1, iter = 7, loss = 10.382, 0s per 1 iters
tensor(10.3913, device='cuda:1')
time = 0m, epoch 1, iter = 8, loss = 10.391, 0s per 1 iters
tensor(10.3743, device='cuda:1')
time = 0m, epoch 1, iter = 9, loss = 10.374, 0s per 1 iters
tensor(10.4088, device='cuda:1')
time = 0m, epoch 1, iter = 10, loss = 10.409, 0s per 1 iters
tensor(10.3906, device='cuda:1')
time = 0m, epoch 1, iter = 11, loss 

tensor(9.9963, device='cuda:1')
time = 0m, epoch 1, iter = 90, loss = 9.996, 0s per 1 iters
tensor(10.0112, device='cuda:1')
time = 0m, epoch 1, iter = 91, loss = 10.011, 0s per 1 iters
tensor(9.9938, device='cuda:1')
time = 0m, epoch 1, iter = 92, loss = 9.994, 0s per 1 iters
tensor(10.0391, device='cuda:1')
time = 0m, epoch 1, iter = 93, loss = 10.039, 0s per 1 iters
tensor(9.9862, device='cuda:1')
time = 0m, epoch 1, iter = 94, loss = 9.986, 0s per 1 iters
tensor(9.9784, device='cuda:1')
time = 0m, epoch 1, iter = 95, loss = 9.978, 0s per 1 iters
tensor(9.9659, device='cuda:1')
time = 0m, epoch 1, iter = 96, loss = 9.966, 0s per 1 iters
tensor(9.9770, device='cuda:1')
time = 0m, epoch 1, iter = 97, loss = 9.977, 0s per 1 iters
tensor(9.9618, device='cuda:1')
time = 0m, epoch 1, iter = 98, loss = 9.962, 0s per 1 iters
tensor(9.9806, device='cuda:1')
time = 0m, epoch 1, iter = 99, loss = 9.981, 0s per 1 iters
tensor(9.9371, device='cuda:1')
time = 0m, epoch 1, iter = 100, loss = 9.937

tensor(9.6153, device='cuda:1')
time = 0m, epoch 1, iter = 180, loss = 9.615, 0s per 1 iters
tensor(9.5409, device='cuda:1')
time = 0m, epoch 1, iter = 181, loss = 9.541, 0s per 1 iters
tensor(9.5907, device='cuda:1')
time = 0m, epoch 1, iter = 182, loss = 9.591, 0s per 1 iters
tensor(9.5209, device='cuda:1')
time = 0m, epoch 1, iter = 183, loss = 9.521, 0s per 1 iters
tensor(9.4610, device='cuda:1')
time = 0m, epoch 1, iter = 184, loss = 9.461, 0s per 1 iters
tensor(9.5349, device='cuda:1')
time = 0m, epoch 1, iter = 185, loss = 9.535, 0s per 1 iters
tensor(9.4724, device='cuda:1')
time = 0m, epoch 1, iter = 186, loss = 9.472, 0s per 1 iters
tensor(9.5585, device='cuda:1')
time = 0m, epoch 1, iter = 187, loss = 9.558, 0s per 1 iters
tensor(9.4932, device='cuda:1')
time = 0m, epoch 1, iter = 188, loss = 9.493, 0s per 1 iters
tensor(9.4075, device='cuda:1')
time = 0m, epoch 1, iter = 189, loss = 9.407, 0s per 1 iters
tensor(9.5574, device='cuda:1')
time = 0m, epoch 1, iter = 190, loss =

tensor(8.9860, device='cuda:1')
time = 0m, epoch 1, iter = 270, loss = 8.986, 0s per 1 iters
tensor(9.0594, device='cuda:1')
time = 0m, epoch 1, iter = 271, loss = 9.059, 0s per 1 iters
tensor(8.9407, device='cuda:1')
time = 0m, epoch 1, iter = 272, loss = 8.941, 0s per 1 iters
tensor(9.0457, device='cuda:1')
time = 0m, epoch 1, iter = 273, loss = 9.046, 0s per 1 iters
tensor(9.2881, device='cuda:1')
time = 0m, epoch 1, iter = 274, loss = 9.288, 0s per 1 iters
tensor(9.1683, device='cuda:1')
time = 0m, epoch 1, iter = 275, loss = 9.168, 0s per 1 iters
tensor(9.1695, device='cuda:1')
time = 0m, epoch 1, iter = 276, loss = 9.169, 0s per 1 iters
tensor(8.9311, device='cuda:1')
time = 0m, epoch 1, iter = 277, loss = 8.931, 0s per 1 iters
tensor(9.1168, device='cuda:1')
time = 0m, epoch 1, iter = 278, loss = 9.117, 0s per 1 iters
tensor(9.0594, device='cuda:1')
time = 0m, epoch 1, iter = 279, loss = 9.059, 0s per 1 iters
tensor(9.1036, device='cuda:1')
time = 0m, epoch 1, iter = 280, loss =

tensor(8.8147, device='cuda:1')
time = 1m, epoch 1, iter = 360, loss = 8.815, 0s per 1 iters
tensor(8.5762, device='cuda:1')
time = 1m, epoch 1, iter = 361, loss = 8.576, 0s per 1 iters
tensor(8.7015, device='cuda:1')
time = 1m, epoch 1, iter = 362, loss = 8.701, 0s per 1 iters
tensor(8.4328, device='cuda:1')
time = 1m, epoch 1, iter = 363, loss = 8.433, 0s per 1 iters
tensor(8.8720, device='cuda:1')
time = 1m, epoch 1, iter = 364, loss = 8.872, 0s per 1 iters
tensor(8.7851, device='cuda:1')
time = 1m, epoch 1, iter = 365, loss = 8.785, 0s per 1 iters
tensor(8.5956, device='cuda:1')
time = 1m, epoch 1, iter = 366, loss = 8.596, 0s per 1 iters
tensor(8.4826, device='cuda:1')
time = 1m, epoch 1, iter = 367, loss = 8.483, 0s per 1 iters
tensor(8.5512, device='cuda:1')
time = 1m, epoch 1, iter = 368, loss = 8.551, 0s per 1 iters
tensor(8.5517, device='cuda:1')
time = 1m, epoch 1, iter = 369, loss = 8.552, 0s per 1 iters
tensor(8.5101, device='cuda:1')
time = 1m, epoch 1, iter = 370, loss =

...

tensor(5.5848, device='cuda:1')
time = 27m, epoch 1, iter = 10373, loss = 5.585, 0s per 1 iters
tensor(4.5359, device='cuda:1')
time = 27m, epoch 1, iter = 10374, loss = 4.536, 0s per 1 iters
tensor(4.6785, device='cuda:1')
time = 27m, epoch 1, iter = 10375, loss = 4.678, 0s per 1 iters
tensor(4.6541, device='cuda:1')
time = 27m, epoch 1, iter = 10376, loss = 4.654, 0s per 1 iters
tensor(4.5050, device='cuda:1')
time = 27m, epoch 1, iter = 10377, loss = 4.505, 0s per 1 iters
tensor(4.2929, device='cuda:1')
time = 27m, epoch 1, iter = 10378, loss = 4.293, 0s per 1 iters
tensor(4.2720, device='cuda:1')
time = 27m, epoch 1, iter = 10379, loss = 4.272, 0s per 1 iters
tensor(4.4230, device='cuda:1')
time = 27m, epoch 1, iter = 10380, loss = 4.423, 0s per 1 iters
tensor(4.4966, device='cuda:1')
time = 27m, epoch 1, iter = 10381, loss = 4.497, 0s per 1 iters
tensor(4.8515, device='cuda:1')
time = 27m, epoch 1, iter = 10382, loss = 4.852, 0s per 1 iters
tensor(5.7060, device='cuda:1')
time = 2

tensor(5.2717, device='cuda:1')
time = 27m, epoch 1, iter = 10459, loss = 5.272, 0s per 1 iters
tensor(4.3457, device='cuda:1')
time = 27m, epoch 1, iter = 10460, loss = 4.346, 0s per 1 iters
tensor(4.4831, device='cuda:1')
time = 27m, epoch 1, iter = 10461, loss = 4.483, 0s per 1 iters
tensor(4.3608, device='cuda:1')
time = 27m, epoch 1, iter = 10462, loss = 4.361, 0s per 1 iters
tensor(4.5477, device='cuda:1')
time = 27m, epoch 1, iter = 10463, loss = 4.548, 0s per 1 iters
tensor(5.3341, device='cuda:1')
time = 27m, epoch 1, iter = 10464, loss = 5.334, 0s per 1 iters
tensor(4.5297, device='cuda:1')
time = 27m, epoch 1, iter = 10465, loss = 4.530, 0s per 1 iters
tensor(4.7278, device='cuda:1')
time = 27m, epoch 1, iter = 10466, loss = 4.728, 0s per 1 iters
tensor(4.6080, device='cuda:1')
time = 27m, epoch 1, iter = 10467, loss = 4.608, 0s per 1 iters
tensor(5.3029, device='cuda:1')
time = 27m, epoch 1, iter = 10468, loss = 5.303, 0s per 1 iters
tensor(4.4638, device='cuda:1')
time = 2

tensor(4.4448, device='cuda:1')
time = 27m, epoch 1, iter = 10545, loss = 4.445, 0s per 1 iters
tensor(4.4441, device='cuda:1')
time = 27m, epoch 1, iter = 10546, loss = 4.444, 0s per 1 iters
tensor(5.1718, device='cuda:1')
time = 27m, epoch 1, iter = 10547, loss = 5.172, 0s per 1 iters
tensor(4.6746, device='cuda:1')
time = 27m, epoch 1, iter = 10548, loss = 4.675, 0s per 1 iters
tensor(4.8298, device='cuda:1')
time = 27m, epoch 1, iter = 10549, loss = 4.830, 0s per 1 iters
tensor(4.2426, device='cuda:1')
time = 27m, epoch 1, iter = 10550, loss = 4.243, 0s per 1 iters
tensor(4.5830, device='cuda:1')
time = 27m, epoch 1, iter = 10551, loss = 4.583, 0s per 1 iters
tensor(4.8728, device='cuda:1')
time = 27m, epoch 1, iter = 10552, loss = 4.873, 0s per 1 iters
tensor(4.8986, device='cuda:1')
time = 27m, epoch 1, iter = 10553, loss = 4.899, 0s per 1 iters
tensor(4.9148, device='cuda:1')
time = 27m, epoch 1, iter = 10554, loss = 4.915, 0s per 1 iters
tensor(4.2412, device='cuda:1')
time = 2

tensor(5.0091, device='cuda:1')
time = 27m, epoch 1, iter = 10631, loss = 5.009, 0s per 1 iters
tensor(5.1963, device='cuda:1')
time = 27m, epoch 1, iter = 10632, loss = 5.196, 0s per 1 iters
tensor(5.1952, device='cuda:1')
time = 27m, epoch 1, iter = 10633, loss = 5.195, 0s per 1 iters
tensor(4.6525, device='cuda:1')
time = 27m, epoch 1, iter = 10634, loss = 4.653, 0s per 1 iters
tensor(4.3131, device='cuda:1')
time = 27m, epoch 1, iter = 10635, loss = 4.313, 0s per 1 iters
tensor(4.6078, device='cuda:1')
time = 27m, epoch 1, iter = 10636, loss = 4.608, 0s per 1 iters
tensor(4.3228, device='cuda:1')
time = 27m, epoch 1, iter = 10637, loss = 4.323, 0s per 1 iters
tensor(4.8639, device='cuda:1')
time = 27m, epoch 1, iter = 10638, loss = 4.864, 0s per 1 iters
tensor(5.9872, device='cuda:1')
time = 27m, epoch 1, iter = 10639, loss = 5.987, 0s per 1 iters
tensor(5.2479, device='cuda:1')
time = 27m, epoch 1, iter = 10640, loss = 5.248, 0s per 1 iters
tensor(5.4682, device='cuda:1')
time = 2

tensor(4.8392, device='cuda:1')
time = 28m, epoch 1, iter = 10717, loss = 4.839, 0s per 1 iters
tensor(4.3626, device='cuda:1')
time = 28m, epoch 1, iter = 10718, loss = 4.363, 0s per 1 iters
tensor(4.4411, device='cuda:1')
time = 28m, epoch 1, iter = 10719, loss = 4.441, 0s per 1 iters
tensor(3.9300, device='cuda:1')
time = 28m, epoch 1, iter = 10720, loss = 3.930, 0s per 1 iters
tensor(4.6985, device='cuda:1')
time = 28m, epoch 1, iter = 10721, loss = 4.698, 0s per 1 iters
tensor(5.2087, device='cuda:1')
time = 28m, epoch 1, iter = 10722, loss = 5.209, 0s per 1 iters
tensor(4.9739, device='cuda:1')
time = 28m, epoch 1, iter = 10723, loss = 4.974, 0s per 1 iters
tensor(5.7110, device='cuda:1')
time = 28m, epoch 1, iter = 10724, loss = 5.711, 0s per 1 iters
tensor(5.3495, device='cuda:1')
time = 28m, epoch 1, iter = 10725, loss = 5.349, 0s per 1 iters
tensor(4.9558, device='cuda:1')
time = 28m, epoch 1, iter = 10726, loss = 4.956, 0s per 1 iters
tensor(4.7400, device='cuda:1')
time = 2